# DATA INSERTION

In [1]:
import pandas as pd
import zipfile

# Define the ZIP file path
zip_file_path = "data_files.zip"  # Replace with the path to your ZIP file

# Open the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # List all files in the ZIP archive (for debugging or verification)
    print("Files in ZIP archive:", zip_ref.namelist())
    
    # Open CSV files directly
    with zip_ref.open("train.csv") as train_file:
        train_data = pd.read_csv(train_file)
    
    with zip_ref.open("test.csv") as test_file:
        test_data = pd.read_csv(test_file)



Files in ZIP archive: ['test.csv', 'train.csv']


C:\Users\Usuario\AppData\Local\Temp\ipykernel_5304\1031665088.py:14: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv(train_file)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_5304\1031665088.py:17: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  test_data = pd.read_csv(test_file)


In [2]:
# Display basic information
print("Train Data:")
print(train_data.info())
print("\nTest Data:")
print(test_data.info())

# Preview the first few rows of the train and test datasets
n = len(train_data)
print(f"Size of the dataset: {n}")




Train Data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107437 entries, 0 to 107436
Data columns (total 55 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   Characteristics.LotFeatures             42020 non-null   object 
 1   Characteristics.LotSizeSquareFeet       1690 non-null    float64
 2   ImageData.c1c6.summary.bathroom         90744 non-null   float64
 3   ImageData.c1c6.summary.exterior         87789 non-null   float64
 4   ImageData.c1c6.summary.interior         93597 non-null   float64
 5   ImageData.c1c6.summary.kitchen          92320 non-null   float64
 6   ImageData.c1c6.summary.property         103055 non-null  float64
 7   ImageData.features_reso.results         104780 non-null  object 
 8   ImageData.q1q6.summary.bathroom         90708 non-null   float64
 9   ImageData.q1q6.summary.exterior         82565 non-null   float64
 10  ImageData.q1q6.summary.interior 

# DATA CLEANING AND PROCESSING

## 1. DROPPING UNNECESSARY COLUMNS 

In [3]:
cleaned_train_data = train_data.copy()
cleaned_train_data = cleaned_train_data.drop(columns=["Location.Address.CensusTract", "Location.School.HighSchoolDistrict", 'Location.Address.PostalCodePlus4', 'Location.Address.UnparsedAddress', 'Location.Address.StreetDirectionPrefix','Location.Address.StreetDirectionSuffix','Location.Address.StreetNumber', 'Characteristics.LotFeatures', 'Listing.Dates.CloseDate', 'Location.Address.StateOrProvince', 'Location.GIS.Latitude', 'Location.GIS.Longitude', 'Location.Address.UnitNumber', 'Characteristics.LotSizeSquareFeet', 'Location.Area.SubdivisionName', 'Structure.BelowGradeFinishedArea', 'Structure.BelowGradeUnfinishedArea', 'Structure.ParkingFeatures', 'Tax.Zoning', 'UnitTypes.UnitTypeType', 'Location.Address.City', 'Location.Address.CountyOrParish', 'ImageData.style.exterior.summary.label', 'Location.Address.StreetName', 'Structure.Heating', 'Location.Address.StreetSuffix'])
print(len(cleaned_train_data.columns))

cleaned_test_data = test_data.copy()
cleaned_test_data = cleaned_test_data.drop(columns=["Location.Address.CensusTract", "Location.School.HighSchoolDistrict", 'Location.Address.PostalCodePlus4', 'Location.Address.UnparsedAddress', 'Location.Address.StreetDirectionPrefix','Location.Address.StreetDirectionSuffix','Location.Address.StreetNumber', 'Characteristics.LotFeatures', 'Listing.Dates.CloseDate', 'Location.Address.StateOrProvince', 'Location.GIS.Latitude', 'Location.GIS.Longitude', 'Location.Address.UnitNumber', 'Characteristics.LotSizeSquareFeet', 'Location.Area.SubdivisionName', 'Structure.BelowGradeFinishedArea', 'Structure.BelowGradeUnfinishedArea', 'Structure.ParkingFeatures', 'Tax.Zoning', 'UnitTypes.UnitTypeType', 'Location.Address.City', 'Location.Address.CountyOrParish', 'ImageData.style.exterior.summary.label', 'Location.Address.StreetName', 'Structure.Heating', 'Location.Address.StreetSuffix'])
print(len(cleaned_test_data.columns))

29
28


### 2. DROPPING ROWS WITH FEW FEATURES DEFINED 


In [4]:
# # Find rows with less than 10 non-null features
# rows_with_few_features = cleaned_train_data[cleaned_train_data.notnull().sum(axis=1) < 20]
# # Print rows with fewer than 10 non-null features (optional)
# print(f"Number of rows with fewer than 20 non-null features: {len(rows_with_few_features)}")
# # Remove these rows from the dataset
# cleaned_train_data = cleaned_train_data[cleaned_train_data.notnull().sum(axis=1) >= 20]
# # Verify the new dataset shape
print(f"Dataset shape after removal: {cleaned_train_data.shape}")

# # Find rows with less than 10 non-null features
# rows_with_few_features = cleaned_test_data[cleaned_test_data.notnull().sum(axis=1) < 20]
# # Print rows with fewer than 10 non-null features (optional)
# print(f"Number of rows with fewer than 20 non-null features: {len(rows_with_few_features)}")
# # Remove these rows from the dataset
# cleaned_test_data = cleaned_test_data[cleaned_test_data.notnull().sum(axis=1) >= 20]
# # Verify the new dataset shape
print(f"Dataset shape after removal: {cleaned_test_data.shape}")

Dataset shape after removal: (107437, 29)
Dataset shape after removal: (22039, 28)


### CONVERTING CATEGORICAL FEATURES TO NUMERICAL AND HANDLING NULL VALUES

In [5]:
# We convert this numerical featuer first because we will need values of a categoricak features, otherwise we'd loose that information
# Convert `Structure.FireplacesTotal` to binary (1 if >0, 0 if 0 or NaN)
cleaned_train_data["Structure.FireplacesTotal"] = cleaned_train_data["Structure.FireplacesTotal"].apply(lambda x: 1 if pd.notnull(x) and x > 0 else 0)
# Identify houses with "fireplace" in `ImageData.features_reso.results`
def has_fireplace(features):
    if isinstance(features, list):  # Ensure it's a list
        return any("fireplace" in feature.lower() for feature in features)
    return False
cleaned_train_data["HasFireplaceFromImage"] = cleaned_train_data["ImageData.features_reso.results"].apply(has_fireplace)
# Update `Structure.FireplacesTotal` based on `HasFireplaceFromImage`
cleaned_train_data["Structure.FireplacesTotal"] = cleaned_train_data.apply(
    lambda row: 1 if row["HasFireplaceFromImage"] and row["Structure.FireplacesTotal"] == 0 else row["Structure.FireplacesTotal"],
    axis=1
)
# Drop the temporary column 
cleaned_train_data.drop(columns=["HasFireplaceFromImage"], inplace=True)

# We convert this numerical featuer first because we will need values of a categoricak features, otherwise we'd loose that information
# Convert `Structure.FireplacesTotal` to binary (1 if >0, 0 if 0 or NaN)
cleaned_test_data["Structure.FireplacesTotal"] = cleaned_test_data["Structure.FireplacesTotal"].apply(lambda x: 1 if pd.notnull(x) and x > 0 else 0)
# Identify houses with "fireplace" in `ImageData.features_reso.results`
def has_fireplace(features):
    if isinstance(features, list):  # Ensure it's a list
        return any("fireplace" in feature.lower() for feature in features)
    return False
cleaned_test_data["HasFireplaceFromImage"] = cleaned_test_data["ImageData.features_reso.results"].apply(has_fireplace)
# Update `Structure.FireplacesTotal` based on `HasFireplaceFromImage`
cleaned_test_data["Structure.FireplacesTotal"] = cleaned_test_data.apply(
    lambda row: 1 if row["HasFireplaceFromImage"] and row["Structure.FireplacesTotal"] == 0 else row["Structure.FireplacesTotal"],
    axis=1
)
# Drop the temporary column 
cleaned_test_data.drop(columns=["HasFireplaceFromImage"], inplace=True)


In [6]:
import ast

# Convert string representations of lists to actual lists
cleaned_train_data['ImageData.features_reso.results'] = cleaned_train_data['ImageData.features_reso.results'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)
# Fill missing values with empty lists
cleaned_train_data['ImageData.features_reso.results'] = cleaned_train_data['ImageData.features_reso.results'].apply(
    lambda x: x if isinstance(x, list) else []
)
# Modify the column to contain the count of features
cleaned_train_data['ImageData.features_reso.results'] = cleaned_train_data['ImageData.features_reso.results'].apply(len)


# Convert string representations of lists to actual lists
cleaned_test_data['ImageData.features_reso.results'] = cleaned_test_data['ImageData.features_reso.results'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)
# Fill missing values with empty lists
cleaned_test_data['ImageData.features_reso.results'] = cleaned_test_data['ImageData.features_reso.results'].apply(
    lambda x: x if isinstance(x, list) else []
)
# Modify the column to contain the count of features
cleaned_test_data['ImageData.features_reso.results'] = cleaned_test_data['ImageData.features_reso.results'].apply(len)


In [7]:
# List of columns to process
columns_to_process = [
    'ImageData.c1c6.summary.bathroom',
    'ImageData.c1c6.summary.exterior',
    'ImageData.c1c6.summary.interior',
    'ImageData.c1c6.summary.kitchen',
    'ImageData.c1c6.summary.property',
    'ImageData.q1q6.summary.bathroom', 
    'ImageData.q1q6.summary.exterior',
    'ImageData.q1q6.summary.interior', 
    'ImageData.q1q6.summary.kitchen',
    'ImageData.q1q6.summary.property'
    
]

# Impute missing values with the median
for col in columns_to_process:    
    median_value = cleaned_train_data[col].median()
    cleaned_train_data[col] = cleaned_train_data[col].fillna(median_value)


# Impute missing values with the median
for col in columns_to_process:    
    cleaned_test_data[col] = cleaned_test_data[col].fillna(median_value) #we use train data for median computation for more accurate value

In [8]:
# Fix for train data
cleaned_train_data['ImageData.room_type_reso.results'] = cleaned_train_data['ImageData.room_type_reso.results'].apply(
    lambda x: len(ast.literal_eval(x)) if isinstance(x, str) else (len(x) if isinstance(x, list) else 0)
)

# Fix for test data
cleaned_test_data['ImageData.room_type_reso.results'] = cleaned_test_data['ImageData.room_type_reso.results'].apply(
    lambda x: len(ast.literal_eval(x)) if isinstance(x, str) else (len(x) if isinstance(x, list) else 0)
)

In [9]:
# Fill missing values within groups by the median of the respective group
cleaned_train_data['Structure.GarageSpaces'] = cleaned_train_data.groupby('Property.PropertyType')['Structure.GarageSpaces'].transform(lambda x: x.fillna(x.median()))
# Fill any remaining NaN values with the global median (if any group had no data to compute the median)
global_median = cleaned_train_data['Structure.GarageSpaces'].median()
cleaned_train_data['Structure.GarageSpaces'].fillna(global_median, inplace=True)


# Fill missing values within groups by the median of the respective group
cleaned_test_data['Structure.GarageSpaces'] = cleaned_test_data.groupby('Property.PropertyType')['Structure.GarageSpaces'].transform(lambda x: x.fillna(x.median()))
# Fill any remaining NaN values with the global median (if any group had no data to compute the median)
cleaned_test_data['Structure.GarageSpaces'].fillna(global_median, inplace=True)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_5304\1341542865.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cleaned_train_data['Structure.GarageSpaces'].fillna(global_median, inplace=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_5304\1341542865.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values

In [10]:
cleaned_train_data = cleaned_train_data.drop(columns='Property.PropertyType')
cleaned_test_data = cleaned_test_data.drop(columns='Property.PropertyType')

In [11]:
# # Apply One-Hot Encoding to 'Property.PropertyType'
# encoded_property_type = pd.get_dummies(cleaned_train_data['Property.PropertyType'], prefix='Property_Type')
# # Concatenate the encoded columns with the original DataFrame
# cleaned_train_data = pd.concat([cleaned_train_data, encoded_property_type], axis=1)
# cleaned_train_data = cleaned_train_data.drop(columns=["Property.PropertyType"])

# # Apply One-Hot Encoding to 'Property.PropertyType'
# encoded_property_type = pd.get_dummies(cleaned_test_data['Property.PropertyType'], prefix='Property_Type')
# # Concatenate the encoded columns with the original DataFrame
# cleaned_test_data = pd.concat([cleaned_test_data, encoded_property_type], axis=1)
# cleaned_test_data = cleaned_test_data.drop(columns=["Property.PropertyType"])

In [12]:
def transform_basement(row):
    if pd.notnull(row['Structure.Basement']):
        if 'none' in row['Structure.Basement']:
            return 0
        return 1
    else:
        # Check if 'ImageData.room_type_reso.results' is not null and is a list
        if pd.notnull(row['ImageData.room_type_reso.results']) and isinstance(row['ImageData.room_type_reso.results'], list):           
            if 'basement' in row['ImageData.room_type_reso.results']:
                return 1
    # If no basement found in both columns, return 0
    return 0
cleaned_train_data['Structure.Basement'] = cleaned_train_data.apply(transform_basement, axis=1)

cleaned_test_data['Structure.Basement'] = cleaned_test_data.apply(transform_basement, axis=1)


In [13]:
# Handle missing values: Impute with median
median_full = cleaned_train_data['Structure.BathroomsFull'].median()
cleaned_train_data['Structure.BathroomsFull'].fillna(median_full, inplace=True)
median_half = cleaned_train_data['Structure.BathroomsHalf'].median()
cleaned_train_data['Structure.BathroomsHalf'].fillna(median_half, inplace=True)
# Create TotalBathrooms feature
cleaned_train_data['TotalBathrooms'] = (
    cleaned_train_data['Structure.BathroomsFull'] +
    0.5 * cleaned_train_data['Structure.BathroomsHalf']
)
# Drop original columns
cleaned_train_data.drop(columns=['Structure.BathroomsFull', 'Structure.BathroomsHalf'], inplace=True)

# Handle missing values: Impute with median
cleaned_test_data['Structure.BathroomsFull'].fillna(median_full, inplace=True)
cleaned_test_data['Structure.BathroomsHalf'].fillna(median_half, inplace=True)
# Create TotalBathrooms feature
cleaned_test_data['TotalBathrooms'] = (
    cleaned_test_data['Structure.BathroomsFull'] +
    0.5 * cleaned_test_data['Structure.BathroomsHalf']
)
# Drop original columns
cleaned_test_data.drop(columns=['Structure.BathroomsFull', 'Structure.BathroomsHalf'], inplace=True)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_5304\2795932034.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cleaned_train_data['Structure.BathroomsFull'].fillna(median_full, inplace=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_5304\2795932034.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values a

In [14]:
# Handle missing values: Impute with median
median_bedrooms = cleaned_train_data['Structure.BedroomsTotal'].median()
cleaned_train_data['Structure.BedroomsTotal'].fillna(median_bedrooms, inplace=True)

# Handle missing values: Impute with median
cleaned_test_data['Structure.BedroomsTotal'].fillna(median_bedrooms, inplace=True)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_5304\1799808172.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cleaned_train_data['Structure.BedroomsTotal'].fillna(median_bedrooms, inplace=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_5304\1799808172.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting valu

In [15]:
def transform_cooling(row):
    if pd.notnull(row['Structure.Cooling']):
        if 'none' in row['Structure.Cooling']:
            return 0
        return 1
    else:
        if pd.notnull(row['ImageData.features_reso.results']) and isinstance(row['ImageData.features_reso.results'], list):           
            if 'cooling' in row['ImageData.features_reso.results']:
                return 1
    # If no basement found in both columns, return 0
    return 0
cleaned_train_data['Structure.Cooling'] = cleaned_train_data.apply(transform_cooling, axis=1)

cleaned_test_data['Structure.Cooling'] = cleaned_test_data.apply(transform_cooling, axis=1)

In [16]:
mean_living_area = cleaned_train_data['Structure.LivingArea'].mean()
cleaned_train_data['Structure.LivingArea'].fillna(mean_living_area, inplace=True)

cleaned_test_data['Structure.LivingArea'].fillna(mean_living_area, inplace=True)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_5304\2073616298.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cleaned_train_data['Structure.LivingArea'].fillna(mean_living_area, inplace=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_5304\2073616298.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values

In [17]:
# Function to impute missing values in 'Structure.NewConstructionYN' based on the year of construction
def impute_new_construction(row):
    if pd.isnull(row['Structure.NewConstructionYN']):
        # Impute based on 'Structure.YearBuilt'
        if row['Structure.YearBuilt'] > 2020:
            return True
        else:
            return False 
    return row['Structure.NewConstructionYN']  # If not null, return the existing value

# Apply the function to impute missing values
cleaned_train_data['Structure.NewConstructionYN'] = cleaned_train_data.apply(impute_new_construction, axis=1)

cleaned_test_data['Structure.NewConstructionYN'] = cleaned_test_data.apply(impute_new_construction, axis=1)

In [18]:
median_year = cleaned_train_data['Structure.YearBuilt'].median()
cleaned_train_data['Structure.YearBuilt'].fillna(median_year, inplace=True)
current_year = 2024
cleaned_train_data['PropertyAge'] = current_year - cleaned_train_data['Structure.YearBuilt']
cleaned_train_data = cleaned_train_data.drop(columns=['Structure.YearBuilt'])

cleaned_test_data['Structure.YearBuilt'].fillna(median_year, inplace=True)
current_year = 2024
cleaned_test_data['PropertyAge'] = current_year - cleaned_test_data['Structure.YearBuilt']
cleaned_test_data = cleaned_test_data.drop(columns=['Structure.YearBuilt'])

C:\Users\Usuario\AppData\Local\Temp\ipykernel_5304\4033238223.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cleaned_train_data['Structure.YearBuilt'].fillna(median_year, inplace=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_5304\4033238223.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alway

In [19]:
# Mapping function to convert story labels to numeric values
def map_stories_to_numeric(value):
    if pd.isnull(value):
        return value  # Keep the NaN value
    elif value == '1_story':
        return 1
    elif value == '2_stories':
        return 2
    elif value == '3_stories_or_more':
        return 3
    elif value == '1.5_stories':
        return 1.5
    elif value == '2.5_stories':
        return 2.5

cleaned_train_data['ImageData.style.stories.summary.label'] = cleaned_train_data['ImageData.style.stories.summary.label'].apply(map_stories_to_numeric)
# Fill null values with the median of the column
median_value = cleaned_train_data['ImageData.style.stories.summary.label'].median()
cleaned_train_data['ImageData.style.stories.summary.label'] = cleaned_train_data['ImageData.style.stories.summary.label'].fillna(median_value)


cleaned_test_data['ImageData.style.stories.summary.label'] = cleaned_test_data['ImageData.style.stories.summary.label'].apply(map_stories_to_numeric)
# Fill null values with the median of the column
cleaned_test_data['ImageData.style.stories.summary.label'] = cleaned_test_data['ImageData.style.stories.summary.label'].fillna(median_value)

In [20]:
cleaned_train_data['Structure.Rooms.RoomsTotal'] = cleaned_train_data['Structure.Rooms.RoomsTotal'].fillna(
    cleaned_train_data['Structure.BedroomsTotal'] + 
    cleaned_train_data['TotalBathrooms'] + 
    2  # Add 1 for common areas (e.g., kitchen, living room)
)

cleaned_test_data['Structure.Rooms.RoomsTotal'] = cleaned_test_data['Structure.Rooms.RoomsTotal'].fillna(
    cleaned_train_data['Structure.BedroomsTotal'] + 
    cleaned_train_data['TotalBathrooms'] + 
    2  # Add 1 for common areas (e.g., kitchen, living room)
)

In [21]:
# from sklearn.preprocessing import MinMaxScaler

# # Step 1: Calculate mean prices by postal code
# postal_code_mean_price = cleaned_train_data.groupby('Location.Address.PostalCode')['Listing.Price.ClosePrice'].mean()

# # Step 2: Normalize the mean prices to a range of 0 to 1
# scaler = MinMaxScaler()
# postal_code_normalized = scaler.fit_transform(postal_code_mean_price.values.reshape(-1, 1))

# # Step 3: Map the normalized scores back to the postal codes
# postal_code_score = pd.DataFrame({
#     'Location.Address.PostalCode': postal_code_mean_price.index,
#     'PostalCode_Score': postal_code_normalized.flatten()
# })

# # Step 4: Merge the scores back into the original DataFrame
# cleaned_train_data = cleaned_train_data.merge(postal_code_score, on='Location.Address.PostalCode', how='left')
# cleaned_train_data = cleaned_train_data.drop(columns=['Location.Address.PostalCode'])


# # Step 4: Merge the scores back into the original DataFrame
# cleaned_test_data = cleaned_test_data.merge(postal_code_score, on='Location.Address.PostalCode', how='left')
# cleaned_test_data = cleaned_test_data.drop(columns=['Location.Address.PostalCode'])

In [22]:
from sklearn.preprocessing import MinMaxScaler
# Step 1: Load the Excel file
zip_income_data = pd.read_excel('Filtered_ZIP_Codes_Total_Income.xlsx')  # Replace with your file path

# Step 2: Create a mapping dictionary
zip_to_income = dict(zip(zip_income_data['ZIP_CODE'], zip_income_data['total_income']))
# Step 3: Map the incomes to the PostalCode column in your DataFrame
cleaned_train_data['Income'] = cleaned_train_data['Location.Address.PostalCode'].map(zip_to_income)
# Step 4: Handle missing ZIP codes
# Replace NaN with the mean or median income of the Excel data
mean_income = zip_income_data['total_income'].mean()
cleaned_train_data['Income'].fillna(mean_income, inplace=True)
# Step 5: Normalize the income column to a score (0-1)
scaler = MinMaxScaler()
cleaned_train_data['Income'] = scaler.fit_transform(cleaned_train_data[['Income']])
cleaned_train_data = cleaned_train_data.drop(columns=["Location.Address.PostalCode"])

# Step 3: Map the incomes to the PostalCode column in your DataFrame
cleaned_test_data['Income'] = cleaned_test_data['Location.Address.PostalCode'].map(zip_to_income)
# Step 4: Handle missing ZIP codes
# Replace NaN with the mean or median income of the Excel data
cleaned_test_data['Income'].fillna(mean_income, inplace=True)
cleaned_test_data['Income'] = scaler.fit_transform(cleaned_test_data[['Income']])
cleaned_test_data = cleaned_test_data.drop(columns=["Location.Address.PostalCode"])

C:\Users\Usuario\AppData\Local\Temp\ipykernel_5304\4270457193.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cleaned_train_data['Income'].fillna(mean_income, inplace=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_5304\4270457193.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves a

In [23]:
from sklearn.preprocessing import MinMaxScaler

# Step 1: Calculate mean prices by postal code
census_block_mean_price = cleaned_train_data.groupby('Location.Address.CensusBlock')['Listing.Price.ClosePrice'].mean()

# Step 2: Normalize the mean prices to a range of 0 to 1
scaler = MinMaxScaler()
census_block_normalized = scaler.fit_transform(census_block_mean_price.values.reshape(-1, 1))

# Step 3: Map the normalized scores back to the postal codes
census_block_score = pd.DataFrame({
    'Location.Address.CensusBlock': census_block_mean_price.index,
    'CensusBlock_Score': census_block_normalized.flatten()
})

# Step 4: Merge the scores back into the original DataFrame
cleaned_train_data = cleaned_train_data.merge(census_block_score, on='Location.Address.CensusBlock', how='left')
cleaned_train_data = cleaned_train_data.drop(columns=['Location.Address.CensusBlock'])


cleaned_test_data = cleaned_test_data.merge(census_block_score, on='Location.Address.CensusBlock', how='left')
cleaned_test_data = cleaned_test_data.drop(columns=['Location.Address.CensusBlock'])

In [24]:
columns_to_convert = ['Structure.NewConstructionYN']

# Convert True/False to 1/0
cleaned_train_data[columns_to_convert] = cleaned_train_data[columns_to_convert].astype(int)

cleaned_test_data[columns_to_convert] = cleaned_test_data[columns_to_convert].astype(int)


In [25]:
cleaned_train_data.to_csv('cleaned_train_data.csv', index=False)
cleaned_test_data.to_csv('cleaned_test_data.csv', index=False)